# 埼玉県　報道発表資料

埼玉県が発表した新型コロナの患者情報のスクレイピングを行います。県内他の市町村から発表された情報は含まれていません。

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#export
import bs4
import re
import csv
import datetime
import requests
from collections import defaultdict

In [3]:
#export
urls = ['https://www.pref.saitama.lg.jp/kense/shiryo/nyu-su/2020/4gatsu/index.html',
        'https://www.pref.saitama.lg.jp/kense/shiryo/nyu-su/2020/3gatsu/index.html',
        'https://www.pref.saitama.lg.jp/kense/shiryo/nyu-su/2020/2gatsu/index.html']

In [4]:
#export
def get_src(url):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    return bs4.BeautifulSoup(response.text, 'html.parser')

In [5]:
#export
def get_links(urls):
    domain = 'https://www.pref.saitama.lg.jp'
    filter = '新型コロナウイルス.+患者.+発生'
    links = []
    for url in urls:
        src = get_src(url)
        links.extend(src.find_all(lambda tag: tag.name == 'a' and re.search(filter, tag.get_text()) != None))
    return [[domain + tag['href'], tag.get_text()] for tag in links]

In [6]:
links = get_links(urls)
links

[['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0412-02.html',
  '新型コロナウイルスに関連した患者等の発生について（県内357～363例目、379～403例目、県発表243例目～274例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0411-01.html',
  '新型コロナウイルスに関連した患者等の発生について（県内339～356例目、県発表225例目～242例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0410-07.html',
  '新型コロナウイルスに関連した患者等の発生について（県内288、299～338例目、県発表184例目～224例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0409-04.html',
  '新型コロナウイルスに関連した患者の発生について（県内252、253例目、266～287例目、県発表160例目～183例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0408-2.html',
  '新型コロナウイルスに関連した患者の発生について（県内229例目～251例目、県発表137例目～159例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0407-04.html',
  '新型コロナウイルスに関連した患者の発生について（県内203例目～207例目、県発表132例目～136例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0406-05.html',
  '新型コロナウイルスに関連した患者等の発生について県内191例目～201例目、県発表121例目～131例目）'],
 ['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0405-01.html',
  

In [7]:
#export
def parse_url(url):
    """ url から公表日を取得する """
    s = re.split("[:/|-]", url)
    return f'{s[7]}/{(s[8])[:2]}/{(s[8])[2:]}'

In [8]:
links[0][0]

'https://www.pref.saitama.lg.jp/a0001/news/page/2020/0412-02.html'

In [9]:
parse_url(links[0][0])

'2020/04/12'

In [10]:
#export
header = ['No','日付','年代','性別','国籍','職業','通勤','居住地','症状、経過','その他','url']

In [11]:
#export
def parse(src, url):
    
    def get_text(tag):
        return tag.get_text().strip()
    
    def get_contents(src):
        h = src.body.find(lambda tag: tag.name == 'h2' or tag.name == 'h3')
        return h.previous_sibling.next_siblings
    
    def parse_h(e):
        no = re.sub("\\D", "", get_text(e))
        return no if len(no) > 0 else '0'

    def parse_p(e):
        return re.split('[：:)）]', get_text(e))
    
    def parse_table(e):
        table = []
        for tr in e.find_all('tr'):
            row = []
            for td in tr.find_all('td'):
                for p in td.find_all('p'):
                    row.append(get_text(p))
            table.append(' '.join(row))
        return table

    def parse_ul(e):
        return [get_text(li) for li in e.find_all('li')]
    
    def parse_div(e):
        ul = []
        for c in e.descendants:
            if c.name == 'p':
                ul.append(get_text(c))
        return ul
    
    def dic2array(item):
        return [item[h] for h in header]

    date = parse_url(url)
    
    patients = []
    item = defaultdict(lambda: '')
    for e in get_contents(src):
        if isinstance(e, bs4.element.Tag):
            if e.name == 'h2' or e.name == 'h3':
                if not '概要' in e.get_text():
                    break
                if len(item) > 0:
                    item['url'] = url
                    item['日付'] = date
                    patients.append(dic2array(item))
                    item = defaultdict(lambda: '')
                item[header[0]] = parse_h(e)
            elif e.name == 'p':
                x = parse_p(e)
                if len(x) > 2: item[x[1]] = x[2]
                if len(x) > 1: p = x[1]
            elif e.name == 'table':
                item[p] = '\n'.join(parse_table(e))
            elif e.name == 'ul':
                item[p] = '\n'.join(parse_ul(e))
            elif e.name == 'div':
                item[p] = '\n'.join(parse_div(e))
            else:
                print(type(e), e.name)
    if len(item) > 0:
        item['url'] = url
        item['日付'] = date
        patients.append(dic2array(item))
    return patients

In [12]:
#export
def get_patient(url):
    src = get_src(url)
    return parse(src, url)

In [13]:
#export
def get_patients(urls):
    patients = []
    for link in get_links(urls):
        print(link)
        patients.extend(get_patient(link[0]))
    return patients

In [14]:
def get_sample(urls, i):
    links = get_links(urls)
    return get_src(links[i][0])
#get_sample(urls, 0)

In [15]:
#export
def create_fname(base):
    now = datetime.datetime.now()
    return base + '_' + now.strftime('%Y%m%dT%H%M') + ".csv"

In [16]:
#export
def write_csv(patients, fname):
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(patients)

In [17]:
#export
def main():
    patients = sorted(get_patients(urls), key=lambda x: int(x[0]))
    patients.insert(0, header)
    fname = create_fname("data/11-pref-saitama")
    write_csv(patients, fname)
    print("created:", fname)

In [18]:
#export
if __name__ == '__main__':
    main()

['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0412-02.html', '新型コロナウイルスに関連した患者等の発生について（県内357～363例目、379～403例目、県発表243例目～274例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0411-01.html', '新型コロナウイルスに関連した患者等の発生について（県内339～356例目、県発表225例目～242例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0410-07.html', '新型コロナウイルスに関連した患者等の発生について（県内288、299～338例目、県発表184例目～224例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0409-04.html', '新型コロナウイルスに関連した患者の発生について（県内252、253例目、266～287例目、県発表160例目～183例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0408-2.html', '新型コロナウイルスに関連した患者の発生について（県内229例目～251例目、県発表137例目～159例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0407-04.html', '新型コロナウイルスに関連した患者の発生について（県内203例目～207例目、県発表132例目～136例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0406-05.html', '新型コロナウイルスに関連した患者等の発生について県内191例目～201例目、県発表121例目～131例目）']
['https://www.pref.saitama.lg.jp/a0001/news/page/2020/0405-01.html', '新型コロナウイルスに関連した患者等の発生について（県内153

In [20]:
# See: https://github.com/fastai/course-v3/blob/master/nbs/dl2/notebook2script.py
!python notebook2script.py 11-pref-saitama_dev.ipynb

Converted 11-pref-saitama_dev.ipynb to exp/nb_11-pref-saitama.py
